In [1]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from chicago_airbnb import df3
import datetime as dt
from config import api_key

importing Jupyter notebook from chicago_airbnb.ipynb


chicago_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
chicago_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
chicago_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [3]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Chicago, IL")

CHI_coord = []
CHI_coord.append(location.latitude)
CHI_coord.append(location.longitude)

CHI_coord

[41.8755616, -87.6244212]

In [4]:
forecast = forecastio.load_forecast(api_key, CHI_coord[0], CHI_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Mixed precipitation throughout the week, with high temperatures bottoming out at 31°F on Monday.
snow
37.13


In [27]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-19',
 '2018-12-20',
 '2018-12-21',
 '2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26']

In [28]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [22]:
df3.dtypes

Date         object
Under $50     int64
$50-100       int64
$100-150      int64
$150-200      int64
$200-250      int64
dtype: object

In [30]:
CHI_table = pd.merge(df, df3, on="Date")
CHI_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-19,49.11,39.98,0.12,0.0001,rain,1066,1806,993,418,167
1,2018-12-20,44.58,32.71,0.74,0.0036,rain,1050,1731,969,430,183
2,2018-12-21,34.55,26.79,0.53,0.0031,snow,877,1521,877,489,255
3,2018-12-22,35.85,28.92,0.06,0.0002,snow,819,1359,800,491,270
4,2018-12-23,33.40,25.15,0.01,0.0001,snow,909,1444,826,459,216
5,2018-12-24,35.16,34.08,0.00,0.0000,none,871,1350,735,448,225
6,2018-12-25,42.76,36.10,0.12,0.0004,rain,869,1334,732,462,221
7,2018-12-26,41.84,41.71,0.97,0.0108,rain,924,1372,777,441,221
